# PyCryption Research Tutorial

This notebook serves a technical introduction, demonstration, and collaboration baseline for this repository, PyCryption. It values 'fire-and-forget' systems to allow you to write, and test custom encryption systems.

> Note: Multi-encryption composition systems are currently under research.

---

## 0.0 - Getting Started: Building a Basic Algorithm

In this section we'll exploring how to rapidly test new encryption algorithms without writing tedious harnesses for testing.

### 0.1 Understand the Mental Model

1. **Define your encryption algorithm into a class with two endpoint functions**: `encrypt`, and `decrypt`, that can accept data, and the algorithm context.
2. **Perform the `quick_test`** to ensure successful dataflow.
3. Register other models and algorithms to compare.
4. Refine and/or repeat.

### 0.2 Encryption Helpers and Algorithm Context

An incredibly powerful component, `AlgorithmContext` is a communication interface to your algorithm from the cryptography components you add to your algorithm.

#### 0.21 Key Injection


In [2]:
from lib.notebook import (
    algorithm,
    with_key,
    generate_key,
    AlgorithmContext,
    ComposerSession,
    quick_test,
)

# -----------------------------------------------------------------------------
# Prototype Algorithm Development
# -----------------------------------------------------------------------------
# Use decorators for logistics (key injection, context, metrics).
# Write your own experimental crypto logic in the methods.

KEY = generate_key(32)


@algorithm("XOR-Prototype")
@with_key(KEY)
class XORPrototype:
    """
    Simple XOR-based prototype for testing the framework.
    NOT cryptographically secure - just a development placeholder.
    """

    def encrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        # Prototype: simple repeating-key XOR
        key = ctx.key
        return bytes(b ^ key[i % len(key)] for i, b in enumerate(data))

    def decrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        # XOR is symmetric
        key = ctx.key
        return bytes(b ^ key[i % len(key)] for i, b in enumerate(data))


# Verify the prototype works
quick_test(XORPrototype())

Testing: XOR-Prototype
Input: b'Hello, PyCryption!'
----------------------------------------
Encrypt: <AlgorithmResult: 18 bytes, 0.022ms>
Decrypt: <AlgorithmResult: 18 bytes, 0.01ms>
Round-trip successful!


In [3]:
# -----------------------------------------------------------------------------
# Composer Session - Benchmark Prototype vs Production Algorithms
# -----------------------------------------------------------------------------
# Register your prototype alongside proven algorithms from lib/algorithms
# to compare performance and validate behavior.

from lib.notebook import wrap_aes256gcm

# Create session and register algorithms
session = ComposerSession()

# Register our prototype
session.register(XORPrototype())

# Register proven algorithm from lib/algorithms (via adapter)
session.register(wrap_aes256gcm(KEY, name="AES-256-GCM"))

print("Registered algorithms:", session.list_algorithms())

# Test all registered algorithms
print("\n=== Round-trip Tests ===")
test_results = session.test_all()
for name, passed in test_results.items():
    status = "PASS" if passed else "FAIL"
    print(f"  {name}: {status}")

Registered algorithms: ['XOR-Prototype', 'AES-256-GCM']

=== Round-trip Tests ===
  XOR-Prototype: PASS
  AES-256-GCM: PASS


In [4]:
# -----------------------------------------------------------------------------
# Benchmark Prototype Performance
# -----------------------------------------------------------------------------

print("=== Prototype Benchmark (10KB, 50 iterations) ===\n")
comparison = session.compare(data_size=10_000, iterations=50)

print(
    f"{'Algorithm':<20} {'Encrypt (ms)':<15} {'Decrypt (ms)':<15} {'Throughput (MB/s)':<15}"
)
print("-" * 65)
for entry in comparison:
    print(
        f"{entry['algorithm']:<20} {entry['avg_encrypt_ms']:<15} {entry['avg_decrypt_ms']:<15} {entry['throughput_mbps']:<15}"
    )

=== Prototype Benchmark (10KB, 50 iterations) ===

Algorithm            Encrypt (ms)    Decrypt (ms)    Throughput (MB/s)
-----------------------------------------------------------------
AES-256-GCM          0.017           0.015           586.85         
XOR-Prototype        1.311           1.363           7.63           


In [5]:
# -----------------------------------------------------------------------------
# Detailed Benchmarks - Scaling Analysis
# -----------------------------------------------------------------------------

print("=== Scaling Analysis ===\n")
benchmarks = session.benchmark_all(
    data_sizes=[100, 1_000, 10_000, 100_000], iterations=20
)

for algo_name, results in benchmarks.items():
    print(f"{algo_name}:")
    print(
        f"  {'Size':<12} {'Encrypt (ms)':<15} {'Decrypt (ms)':<15} {'Throughput':<15}"
    )
    print(f"  {'-' * 55}")
    for bench in results["benchmarks"]:
        size_label = f"{bench['size_bytes']:,} B"
        print(
            f"  {size_label:<12} {bench['avg_encrypt_ms']:<15} {bench['avg_decrypt_ms']:<15} {bench['throughput_mbps']:<15} MB/s"
        )

=== Scaling Analysis ===

XOR-Prototype:
  Size         Encrypt (ms)    Decrypt (ms)    Throughput     
  -------------------------------------------------------
  100 B        0.018           0.017           5.63            MB/s
  1,000 B      0.143           0.149           7.02            MB/s
  10,000 B     1.352           1.328           7.4             MB/s
  100,000 B    9.129           9.155           10.95           MB/s
AES-256-GCM:
  Size         Encrypt (ms)    Decrypt (ms)    Throughput     
  -------------------------------------------------------
  100 B        0.01            0.007           9.71            MB/s
  1,000 B      0.007           0.007           143.88          MB/s
  10,000 B     0.016           0.015           630.91          MB/s
  100,000 B    0.023           0.021           4376.37         MB/s


In [6]:
# -----------------------------------------------------------------------------
# Session Metrics - Aggregated Statistics
# -----------------------------------------------------------------------------

print("=== Session Report ===\n")
report = session.report()

for algo_name, metrics in report.items():
    print(f"{algo_name}:")
    print(
        f"  Operations: {metrics['encrypt_calls']} encrypt, {metrics['decrypt_calls']} decrypt"
    )
    print(
        f"  Avg timing: {metrics['avg_encrypt_ms']:.3f}ms encrypt, {metrics['avg_decrypt_ms']:.3f}ms decrypt"
    )
    print(f"  Total data: {metrics['total_bytes_processed']:,} bytes")
    if metrics["errors"] > 0:
        print(f"  Errors: {metrics['errors']}")
    print()

=== Session Report ===

XOR-Prototype:
  Operations: 1 encrypt, 1 decrypt
  Avg timing: 0.015ms encrypt, 0.006ms decrypt
  Total data: 36 bytes

AES-256-GCM:
  Operations: 1 encrypt, 1 decrypt
  Avg timing: 1.533ms encrypt, 0.039ms decrypt
  Total data: 52 bytes



---

## 1.0: Example – Adding Cryptographic Capabilities

Beyond a simple byte emitter in the last example, we can easily add new capabilities. Let's take a look at the [Salsa20](https://en.wikipedia.org/wiki/Salsa20) algorithm which was succeeded by its descendent, ChaCha in 2008.


In [7]:
import hashlib
from Crypto.Cipher import Salsa20
from lib.notebook import with_metrics


def salsa_kdf(key: bytes, salt: bytes) -> bytes:
    return hashlib.pbkdf2_hmac('sha256', key, salt, 100000, dklen=32)


@algorithm("Salsa20-Prototype")
@with_key(KEY)
@with_metrics()
class Salsa20Prototype:

    def encrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        # Register KDF and salt, derive key, cache result
        ctx.set_kdf("salsa-kdf", salsa_kdf)
        ctx.set_salt("salsa-salt")
        derived = ctx.derive("salsa-kdf", "salsa-salt", cache_as="salsa-derived")


        cipher = Salsa20.new(derived)
        ctx.set_nonce("salsa-nonce", cipher.nonce)
        return cipher.encrypt(data)

    def decrypt(self, data: bytes, ctx: AlgorithmContext) -> bytes:
        # Retrieve cached materials from registry
        derived = ctx.get_derived_key("salsa-derived")
        nonce = ctx.get_nonce("salsa-nonce")
        cipher = Salsa20.new(derived, nonce)
        return cipher.decrypt(data)


# Verify the prototype works
quick_test(Salsa20Prototype())

Testing: Salsa20-Prototype
Input: b'Hello, PyCryption!'
----------------------------------------
Encrypt: <AlgorithmResult: 18 bytes, 40.141ms>
Decrypt: <AlgorithmResult: 18 bytes, 0.033ms>
Round-trip successful!
